# MAGYC Factor Graph-based Approaches Implementation - Custom Factors

## Residual Model Factor


This section aim to get the Jacobian of the residual of the sensor model using the symbolic mathematics libraries available in python, particularly `sympy`. In the first section we will show the residual equation and get the dimension of each of the terms to define the necessary variables. Then, we will use that to compute the Jacobian, i.e., the model differentiated by each one of the variables of the system.

### Sensor Model Residual

For the proposes method, we started with the full magnetometer model that takes in consideration the bias, or hard-iron, and the scale and non-orthogonality represented in the soft-iron matrix. The hard-iron is a column vector in $\mathbb{R}^3$ that we will represent by the letter $b$, while the soft-iron is a matrix in $\mathbb{R}^{3\times3}$ with the property that is a positive definite symmetric matrix (PDS) that we will represent with the letter $A$.

This method also takes into consideration the gyroscope measurements to compute the residuals. The gyroscope measurements are modeled as a ground truth measurement plus a bias that is a column vector in $\mathbb{R}^3$ that we will represent by the letter $b$.

$$ b = \begin{bmatrix}b_0 \\ b_1 \\ b_2\end{bmatrix} \qquad \qquad A = \begin{bmatrix}a_0 & a_1 & a_2 \\ a_1 & a_3 & a_4 \\ a_2 & a_4 & a_5\end{bmatrix} \qquad \qquad d = \begin{bmatrix}d_0 \\ d_1 \\ d_2\end{bmatrix}$$

All this parameters are combined to get the model residuals for each sample $i, \; \forall \; i \in \{1, \dots, n\}$:

$$R_i = [w_i(t)]A^{-1}m_i(t) - [d]A^{-1}m_i(t) + A^{-1}\dot{m}_i(t) - [w_i(t)]b + [d]b $$

Where $[w_i(t)]$ is the $so(3)$ skew-symmetric matrix in $\mathbb{R}^{3\times3}$ for the i-th gyroscope sample, $m_i(t)$ is the i-th magnetometer sample represented as a column vector in $\mathbb{R}^3$, and $\dot{m}_i(t)$ is the corresponding derivative of the magnetic field, also represented as a column vector in $\mathbb{R}^3$, for notation now onwards we will use $\dot{m}_i(t) = n_i(t)$.

$$ [w_i(t)] = \begin{bmatrix} 0 & -w_{i2}(t) & w_{i1}(t) \\ w_{i2}(t) & 0 & -w_{i0}(t) \\ -w_{i1}(t) & w_{i0}(t) & 0\end{bmatrix} \qquad \qquad m_i(t) = \begin{bmatrix}m_{i0}(t) \\ m_{i1}(t) \\ m_{i2}(t)\end{bmatrix} \qquad \qquad n_i(t) = \begin{bmatrix}n_{i0}(t) \\ n_{i1}(t) \\ n_{i2}(t)\end{bmatrix}$$

Now, we can define the variables using `sympy`

In [ ]:
import sympy as sp
from sympy.physics.quantum import TensorProduct as KroneckerProduct
sp.init_printing()

from sympy.printing.numpy import NumPyPrinter
numpy_printer = NumPyPrinter()

In [ ]:
# Magnetometer Symbols
m, mt, m0, m1, m2 = sp.symbols("m, m^t, m_0, m_1, m_2")  # Magnetic field measurement
n, nt, n0, n1, n2 = sp.symbols("n, n^t, n_0, n_1, n_2")  # Magnetic field measurement differentiated w.r.t time
b, b0, b1, b2 = sp.symbols("b, b_0, b_1, b_2")  # Magnetometers hard-iron
a, a0, a1, a2, a3, a4, a5 = sp.symbols("a, a_0, a_1, a_2, a_3, a_4, a_5") # Magnetometer soft-iron

# Magnetometer Matrices and vectors
m = sp.Matrix([[m0], [m1], [m2]])
mt = m.transpose()
n = sp.Matrix([[n0], [n1], [n2]])
nt = n.transpose()
b = sp.Matrix([[b0], [b1], [b2]])
a = sp.Matrix([[a0, a1, a2], [a1, a3, a4], [a2, a4, a5]])

# Gyroscope Symbols
w0, w1, w2 = sp.symbols("w_0, w_1, w_2")  # Gyroscope measurement
d0, d1, d2 = sp.symbols("d_0, d_1, d_2")  # Gyroscope bias

# Gyroscope Matrices and vectors
w = sp.Matrix([[w0], [w1], [w2]])
d = sp.Matrix([[d0], [d1], [d2]])

We can notice that, as the matrix $A$ is positive definite symmetric, then the matrix $C$ is also symmetric. By definition, if we have $(A^{-1})^T = (A^T)^{-1}$ for any invertible matrix. It follows from this that if $A$ is invertible and symmetric: $(A^{-1})^T = (A^T)^{-1} = A^{-1}$; so $A^{-1}$ is also symmetric. Further, if all eigenvalues of $A$ are positive, then $A^{-1}$ exists and all eigenvalues of $A^{-1}$ are positive since they are the reciprocals of the eigenvalues of $A$. Thus $A^{-1}$ is positive definite when $A$ is positive definite.

As $A$ is a PDS matrix, it can be represented by the Cholesky decomposition, i.e., $A = LL^T$, where $L$ is a lower triangular matrix. And further, as $A^{-1}$ is also a PDS matrix, it can be represented by the Cholesky decomposition, i.e., $C = LL^T$, where $C = A^{-1}$.

Also, to ensure positive definitiveness, we can parametrize the diagonal elements of $L$ as exponential terms.

In [ ]:
l, lt, l0, l1, l2, l3, l4, l5 = sp.symbols("l, l^t, l_0, l_1, l_2, l_3, l_4, l_5")
l = sp.Matrix([[sp.exp(l0), 0.0, 0.0], [l1, sp.exp(l2), 0.0], [l3, l4, sp.exp(l5)]])
lt = l.transpose()

Then, the model residual can be represented as:

$$R_i = [w_i(t) - d]LL^{T}(m_i(t) - b) + LL^{T}\dot{m}_i(t) $$

Where $LL^T$ is:

In [ ]:
l @ lt

### Jacobian of the Residual

In [ ]:
# Skew-symmetric matrix
skewd, skeww = sp.symbols("skewd, skeww")
skeww = sp.Matrix([[0, -w2, w1], [w2, 0, -w0], [-w1, w0, 0]])
skewd = sp.Matrix([[0, -d2, d1], [d2, 0, -d0], [-d1, d0, 0]])

# Residual model
r = sp.Function("r")(l, b, d)
r = (skeww - skewd) @ (l @ lt) @ (m - b) + ((l @ lt) @ n)

#### $\frac{\partial r}{\partial l_0}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l0))

#### $\frac{\partial r}{\partial l_1}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l1))

#### $\frac{\partial r}{\partial l_2}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l2))

#### $\frac{\partial r}{\partial l_3}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l3))

#### $\frac{\partial r}{\partial l_4}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l4))

#### $\frac{\partial r}{\partial l_5}$

In [ ]:
numpy_printer.doprint(sp.diff(r, l5))

#### $\frac{\partial r}{\partial b_0}$

In [ ]:
numpy_printer.doprint(sp.diff(r, b0))

#### $\frac{\partial r}{\partial b_1}$

In [ ]:
numpy_printer.doprint(sp.diff(r, b1))

#### $\frac{\partial r}{\partial b_2}$

In [ ]:
numpy_printer.doprint(sp.diff(r, b2))

#### $\frac{\partial r}{\partial d_0}$

In [ ]:
numpy_printer.doprint(sp.diff(r, d0))

#### $\frac{\partial r}{\partial d_1}$

In [ ]:
numpy_printer.doprint(sp.diff(r, d1))

#### $\frac{\partial r}{\partial d_2}$

In [ ]:
numpy_printer.doprint(sp.diff(r, d2))

## Scale Constraint Factor

To constraint the scale of the soft-iron matrix, we can constraint the volume of the ellipsoid defined by the soft-iron matrix to remain constant and equal to the volume of a unitary sphere. The volume of an ellipsoid is given by the formula:

$$V = \frac{4}{3}\pi\lambda_0\lambda_1\lambda_2$$

Where $\lambda_0$, $\lambda_1$, and $\lambda_2$ are the eigenvalues of the soft-iron matrix $A$. The volume of a unitary sphere is given by the formula:

$$V = \frac{4}{3}\pi$$

Then, for a PSD matrix, the determinant is the product of the eigenvalues, i.e., $\det(A) = \lambda_0\lambda_1\lambda_2$. Then, the scale constraint can be represented as:

$$\det(A) = 1$$

Now, lets compute the determinant for the $C = LL^T$ matrix:

In [ ]:
sp.det(l @ lt)

### Jacobian of the Scale Constraint

In [ ]:
# Scale constraint
s = sp.Function("s")(l)
s = sp.det(l @ lt)

#### $\frac{\partial S}{\partial l_0}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l0))

#### $\frac{\partial S}{\partial l_1}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l1))

#### $\frac{\partial S}{\partial l_2}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l2))

#### $\frac{\partial S}{\partial l_3}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l3))

#### $\frac{\partial S}{\partial l_4}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l4))

#### $\frac{\partial S}{\partial l_5}$

In [ ]:
numpy_printer.doprint(sp.diff(s, l5))

## Diagonal Terms Difference Factor

The previous constraint will constraint the product of the eigenvalues to be equal to 1. However, if one of the value tends to zero and another tends to infinity, the product will still be 1. To avoid this, we can constraint the difference between the eigenvalues to be equal to 1. This can be represented as:

We know that the diagonal terms are:

$$C_{00} = exp(2l_0), \quad C_{11} = l_1^2 + exp(2l_2), \quad C_{22} = l_3^2 + l_4^2 + exp(2l_5)$$

Then, the factor residual can be represented as:

In [ ]:
((l @ lt)[0, 0] - (l @ lt)[1, 1])**2 + ((l @ lt)[1, 1] - (l @ lt)[2, 2])**2 + ((l @ lt)[0, 0] - (l @ lt)[2, 2])**2

### Jacobian of the Diagonal Terms Difference

In [ ]:
# Scale constraint
dd = sp.Function("dd")(l)
dd = ((l @ lt)[0, 0] - (l @ lt)[1, 1])**2 + ((l @ lt)[1, 1] - (l @ lt)[2, 2])**2 + ((l @ lt)[0, 0] - (l @ lt)[2, 2])**2

#### $\frac{\partial D}{\partial l_0}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l0)))

#### $\frac{\partial D}{\partial l_1}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l1)))

#### $\frac{\partial D}{\partial l_2}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l2)))

#### $\frac{\partial D}{\partial l_3}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l3)))

#### $\frac{\partial D}{\partial l_4}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l4)))

#### $\frac{\partial D}{\partial l_5}$

In [ ]:
numpy_printer.doprint(sp.simplify(sp.diff(dd, l5)))